# 물류 유통량 예측

# 평가방식 : RMSE

# 데이터 설명

**train.csv**

index : 인덱스

송하인_격자공간고유번호 

수하인 격자공간고유번호

택배_카테고리

운송장_건수 (타겟 데이터)

**test_df.csv**

index : 인덱스

송하인_격자공간고유번호

수하인 격자공간고유번호

택배_카테고리

---

송하인 : 운송 계약에서, 물품의 운송을 맡기는 사람.

수하인 : 운송된 물품을 인도받는 사람.

격자공간고유번호 : 공간정보 표준화를 위해 고유로 부여한 격자공간 고유번호

송하인과 수하인의 경우 16자리의 숫자중 앞의 4자리가 격자 번호이다.

In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
train

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3
...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,스포츠잡화,3
31680,31680,1129000014045300,5011000319087100,스마트디바이스,4
31681,31681,1129000014045300,5011000263065200,스마트디바이스,6
31682,31682,4127300065073100,5011000264061200,지갑,7


In [2]:
test = pd.read_csv('test.csv')
test

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,4167000577042200,5011000435014100,선케어
1,1,1156000009012200,5011000172034400,구강위생용품
2,2,4122000363057300,5011000361097300,캠핑
3,3,5011000436041400,2826000084036400,아웃도어가구
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식
...,...,...,...,...
7915,7915,5011000266051200,4623000417038100,농산물
7916,7916,1154500001098300,5011000264055100,문화컨텐츠
7917,7917,5013000610049100,1147000018091400,농산물
7918,7918,5013000610049100,3117000039026100,농산물


https://www.code.go.kr/stdcode/regCodeL.do

https://namu.wiki/w/%EB%B2%95%EC%A0%95%EB%8F%99/%EB%AA%A9%EB%A1%9D/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C

위 사이트를 통해 법정동 코드로 격자공간고유번호가 어디지역에서 어디지역으로 이동했는지 알 수 있다

앞에 2자리만 필요하고 뒤에 숫자들은 다 버려줄 것이다 또한 index 칼럼도 다 없애버릴것이다.

# 결측치 처리

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         31684 non-null  int64 
 1   송하인_격자공간고유번호  31684 non-null  int64 
 2   수하인_격자공간고유번호  31684 non-null  int64 
 3   물품_카테고리       31684 non-null  object
 4   운송장_건수        31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         7920 non-null   int64 
 1   송하인_격자공간고유번호  7920 non-null   int64 
 2   수하인_격자공간고유번호  7920 non-null   int64 
 3   물품_카테고리       7920 non-null   object
dtypes: int64(3), object(1)
memory usage: 247.6+ KB


결측값은 존재하지 않고 물품 카테고리만 오브젝트형이니 처리해주면 될 것이다..

# 법정동 목록 

서울 : 11

부산 : 26

대구 : 27

인천 : 28

광주 : 29

대전 : 30

울산 : 31

세종 : 36

경기 : 41

강원 : 42

충북 : 43

충남 : 44

전북 : 45

전남 : 46

경북 : 47

경남 : 48

제주 : 50

이외 : 기타

In [5]:
train.drop(['index'], axis = 1)
test.drop(['index'], axis = 1)

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,4167000577042200,5011000435014100,선케어
1,1156000009012200,5011000172034400,구강위생용품
2,4122000363057300,5011000361097300,캠핑
3,5011000436041400,2826000084036400,아웃도어가구
4,4150000241065200,5011000169044300,분유/이유식/아기간식
...,...,...,...
7915,5011000266051200,4623000417038100,농산물
7916,1154500001098300,5011000264055100,문화컨텐츠
7917,5013000610049100,1147000018091400,농산물
7918,5013000610049100,3117000039026100,농산물


In [6]:
train = train.rename(columns = {'송하인_격자공간고유번호':'출발지','수하인_격자공간고유번호':'도착지','물품_카테고리':'카테고리'})
test = test.rename(columns = {'송하인_격자공간고유번호':'출발지','수하인_격자공간고유번호':'도착지','물품_카테고리':'카테고리'})

In [7]:
train = train.astype(str)
train['출발지'] = train['출발지'].str.slice(start=0,stop=2)
train['도착지'] = train['도착지'].str.slice(start=0,stop=2)

test = test.astype(str)
test['출발지'] = test['출발지'].str.slice(start=0,stop=2)
test['도착지'] = test['도착지'].str.slice(start=0,stop=2)

In [8]:
train['출발지'] = train['출발지'].apply(pd.to_numeric)
train['도착지'] = train['도착지'].apply(pd.to_numeric)
train['운송장_건수'] = train['운송장_건수'].apply(pd.to_numeric)

test['출발지'] = test['출발지'].apply(pd.to_numeric)
test['도착지'] = test['도착지'].apply(pd.to_numeric)

In [9]:
train = train.drop(['index'],axis=1)
test = test.drop(['index'],axis=1)

트레인과 테스트 데이터 모두 앞에 2자리 지역번호만 추출해주었고 index 칼럼은 제거해주었다.

이제 각 지역번호에 지역을 대입해주자

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   출발지     31684 non-null  int64 
 1   도착지     31684 non-null  int64 
 2   카테고리    31684 non-null  object
 3   운송장_건수  31684 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 990.2+ KB


In [11]:
def get_category(num):
    area = ''
    if num == 11 : area = '서울'
    elif num == 26 : area = '부산'
    elif num == 27 : area = '대구'
    elif num == 28 : area = '인천'
    elif num == 29 : area = '광주'
    elif num == 30 : area = '대전'
    elif num == 31 : area = '울산'
    elif num == 36 : area = '세종'
    elif num == 41 : area = '경기'
    elif num == 42 : area = '강원'
    elif num == 43 : area = '충북'
    elif num == 44 : area = '충남'
    elif num == 45 : area = '전북'
    elif num == 46 : area = '전남'
    elif num == 47 : area = '경북'
    elif num == 48 : area = '경남'
    elif num == 50 : area = '제주'
    else : area  = '기타'
        
    return area

In [12]:
train['출발지'] = train['출발지'].apply(lambda x : get_category(x))
train['도착지'] = train['도착지'].apply(lambda x : get_category(x))
test['출발지'] = test['출발지'].apply(lambda x : get_category(x))
test['도착지'] = test['도착지'].apply(lambda x : get_category(x))

In [13]:
test

,출발지,도착지,카테고리
0,경기,제주,선케어
1,서울,제주,구강위생용품
2,경기,제주,캠핑
3,제주,인천,아웃도어가구
4,경기,제주,분유/이유식/아기간식
...,...,...,...
7915,제주,전남,농산물
7916,서울,제주,문화컨텐츠
7917,제주,서울,농산물
7918,제주,울산,농산물


In [14]:
train['출발지'].value_counts()

제주    23461
경기     4779
서울     1148
대전      264
인천      245
충북      234
대구      229
부산      226
전북      220
충남      211
경북      198
경남      130
전남      120
강원      118
광주       65
울산       29
세종        7
Name: 출발지, dtype: int64

출발지와 도착지가 잘 변경된것을 볼 수 있다.

In [15]:
train['카테고리'].unique()

array(['음반', '문화컨텐츠', '농산물', '기타식품', '수산', '아우터', '신발', '건강식품', '음료',
       '가공식품', '스킨케어', '축산', '헤어케어', '기저귀/물티슈', '의료용품', '기타디지털/가전', '상의',
       '건강용품', '바디케어', '기타스포츠/레저', '출산/유아동의류', '반려동물', '완구/매트', '과자',
       '생활용품', '하의', '문구/사무용품', '선케어', '다이어트식품', '기타패션의류', '냉동/간편조리식품',
       '네일케어', '서재/사무용가구', '선글라스/안경테', '기타화장품/미용', '등산', '모니터', '주방용품',
       '색조메이크업', '침구세트', '침구단품', '인테리어소품', '양말/스타킹', '클렌징', '공구', '이미용가전',
       'PC', '커튼/블라인드', '헤어스타일링', '주방가구', 'DIY자재/용품', '헤어액세서리', '김치',
       '위생/건강용품', '침실가구', '가방', '주방가전', '낚시', '재활운동용품', '자동차용품', '뷰티소품',
       '주얼리', '골프', 'PC주변기기', '수납/정리용품', '음향가전', '잠옷', '패션소품', '언더웨어',
       '원피스/점프슈트', '스포츠잡화', '수납가구', '취미용품', '아웃도어가구', '계절가전', '홈데코', '캠핑',
       '남성화장품', '분유/이유식/아기간식', '베이스메이크업', '헬스', '향수', '모자', '카페트/러그',
       '구강위생용품', '눈관리용품', '기타출산/육아', '기타패션잡화', '기능성', '반찬', '욕실용품',
       '스마트디바이스액세서리', '게임기/타이틀', '생활가전', '유아가구', '출산/유아동잡화', '세탁용품',
       '태블릿PC/노트북액세서리', '스마트디바이스', '지갑'], dtype=object)

이번엔 카테고리를 정리해보려고 했는데 너무 많다 이걸 일일히 처리하려면 머리아플거같으니 원핫인코딩 해주자...ㅜㅜ

사실 데이터가 한번 바뀐것 같은데 바뀌기 전 데이터는 대분류 소분류로 나뉘어져있어 처리하기 더 깔끔했지만 카테고리가 묶이면서 eda 나 데이터를 처리하기 힘들어졌다

In [18]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [20]:
train

,운송장_건수,출발지_강원,출발지_경기,출발지_경남,출발지_경북,출발지_광주,출발지_대구,출발지_대전,출발지_부산,출발지_서울,...,카테고리_클렌징,카테고리_태블릿PC/노트북액세서리,카테고리_패션소품,카테고리_하의,카테고리_향수,카테고리_헤어스타일링,카테고리_헤어액세서리,카테고리_헤어케어,카테고리_헬스,카테고리_홈데코
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31679,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31680,4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
31681,6,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
31682,7,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


135개의 칼럼이 생성되었다.

모델 생성을 해주자

# 모델 생성

In [28]:
features = train.drop('운송장_건수',axis=1)
target = train['운송장_건수']

In [30]:
import xgboost 
import lightgbm 

In [36]:
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

x_train, x_test, y_train, y_test = train_test_split(features,target)

xgb = XGBRegressor(n_estimators = 300)
lgbm = LGBMRegressor(n_estimators = 300)

In [37]:
xgb_parameter = {'max_depth':[3,5,7], 'min_child_weight':[1,3,5],'colsample_bytree':[0.5,0.75,1]}
lgbm_parameter = {'num_leaves':[32,64],'max_depth':[32,64,128],'min_child_samples':[32,64,128]}

In [39]:
grid_xgb = GridSearchCV(xgb,param_grid = xgb_parameter, cv=3)
grid_xgb.fit(x_train,y_train,eval_metric = "rmse")

print(grid_xgb.best_params_)

{'colsample_bytree': 0.75, 'max_depth': 3, 'min_child_weight': 5}


In [40]:
grid_lgbm = GridSearchCV(lgbm,param_grid = lgbm_parameter, cv=3)
grid_lgbm.fit(x_train,y_train,eval_metric = "rmse")

print(grid_lgbm.best_params_)

{'max_depth': 32, 'min_child_samples': 32, 'num_leaves': 32}


In [42]:
xgb = XGBRegressor(colsample_bytree= 0.75, max_depth= 3, min_child_weight= 5)
lgbm = LGBMRegressor(max_depth=32 ,num_leaves= 32,min_child_smaples=32)

# 모델 학습

In [80]:
xgb.fit(features,target)
lgbm.fit(features,target)
xgb_pred = xgb.predict(test)
lgbm_pred = lgbm.predict(test)

pred = 0.5*xgb_pred + 0.5*lgbm_pred

[LightGBM] [Warning] Unknown parameter: min_child_smaples


In [48]:
sample = pd.read_csv('sample_submission.csv')

In [51]:
sample['운송장_건수']=pred
sample.to_csv('1.csv',index = False)

제출 결과 : 6.02 퍼블릭 225..

---

결과가 생각보다 안좋았다 접근을 잘못한것일까? ...

곰곰히 생각해보니 칼럼의 수가 너무 많은거 같기도 했고...

그렇다면 차원축소를???

# PCA

In [68]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
features_pca = pca.fit_transform(features)

In [77]:
test_pca = pca.fit_transform(test)

In [70]:
features_pca = pd.DataFrame(features_pca)

In [72]:
x_train, x_test, y_train, y_test = train_test_split(features_pca,target)

In [73]:
grid_xgb = GridSearchCV(xgb,param_grid = xgb_parameter, cv=3)
grid_xgb.fit(x_train,y_train,eval_metric = "rmse")

print(grid_xgb.best_params_)

{'colsample_bytree': 0.5, 'max_depth': 3, 'min_child_weight': 5}


In [74]:
grid_lgbm = GridSearchCV(lgbm,param_grid = lgbm_parameter, cv=3)
grid_lgbm.fit(x_train,y_train,eval_metric = "rmse")

print(grid_lgbm.best_params_)

[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Warning] Unknown parameter: min_child_smaples
[LightGBM] [Wa

In [81]:
xgb = XGBRegressor(colsample_bytree=0.5 , max_depth=3 , min_child_weight=5 )
lgbm = LGBMRegressor(max_depth=32 ,num_leaves=128 ,min_child_smaples=64)

In [82]:
xgb.fit(features_pca,target)
lgbm.fit(features_pca,target)
xgb_pred = xgb.predict(test_pca)
lgbm_pred = lgbm.predict(test_pca)

pred_pca = 0.5*xgb_pred + 0.5*lgbm_pred

[LightGBM] [Warning] Unknown parameter: min_child_smaples


In [83]:
sample['운송장_건수']=pred_pca
sample.to_csv('2.csv',index = False)

public : 6.3으로 더 떨어졌다 아마 내가 방향성을 잘못 잡은것 같다.

문자형 데이터를 바로 원핫인코딩하여 여러개의 칼럼을 생성하기 보다는 

문자열 데이터에 숫자를 부여하는 방식으로 했으면? 좀 더 나은 결과가 있었을지도..ㅠㅠ

---